In [22]:
from IPython.display import Audio
import librosa.display


In [2]:
# from keras.models import load_model
# model = load_model('Noise-Suppression-Using-Deep-Learning/Results/models/NoiseSuppressionModel_Model2.h5')


In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D,Conv1DTranspose,Concatenate,Input

def create_model():
    batching_size = 12000
    inp = Input(shape=(batching_size,1))
    c1 = Conv1D(2,32,2,'same',activation='relu')(inp)
    c2 = Conv1D(4,32,2,'same',activation='relu')(c1)
    c3 = Conv1D(8,32,2,'same',activation='relu')(c2)
    c4 = Conv1D(16,32,2,'same',activation='relu')(c3)
    c5 = Conv1D(32,32,2,'same',activation='relu')(c4)

    dc1 = Conv1DTranspose(32,32,1,padding='same')(c5)
    conc = Concatenate()([c5,dc1])
    dc2 = Conv1DTranspose(16,32,2,padding='same')(conc)
    conc = Concatenate()([c4,dc2])
    dc3 = Conv1DTranspose(8,32,2,padding='same')(conc)
    conc = Concatenate()([c3,dc3])
    dc4 = Conv1DTranspose(4,32,2,padding='same')(conc)
    conc = Concatenate()([c2,dc4])
    dc5 = Conv1DTranspose(2,32,2,padding='same')(conc)
    conc = Concatenate()([c1,dc5])
    dc6 = Conv1DTranspose(1,32,2,padding='same')(conc)
    conc = Concatenate()([inp,dc6])
    dc7 = Conv1DTranspose(1,32,1,padding='same',activation='linear')(conc)
    model = tf.keras.models.Model(inp,dc7)
    model.summary()
    return model


model = create_model()
model.load_weights('Noise-Suppression-Using-Deep-Learning/Results/models/NoiseSuppressionModel_Model2.h5')



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 12000, 1)]   0           []                               
                                                                                                  
 conv1d_5 (Conv1D)              (None, 6000, 2)      66          ['input_2[0][0]']                
                                                                                                  
 conv1d_6 (Conv1D)              (None, 3000, 4)      260         ['conv1d_5[0][0]']               
                                                                                                  
 conv1d_7 (Conv1D)              (None, 1500, 8)      1032        ['conv1d_6[0][0]']               
                                                                                              

# Testing Model on the existing dataset

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D,Conv1DTranspose,Concatenate,Input
import numpy as np
import IPython.display
import glob
from tqdm.notebook import tqdm
import librosa.display
import matplotlib.pyplot as plt

noisy_sounds = glob.glob('Noise-Suppression-Using-Deep-Learning/dataset/NoisyData/*')
noisy_sounds = noisy_sounds[:100]

noisy_sounds_list,_ = tf.audio.decode_wav(tf.io.read_file(noisy_sounds[0]),desired_channels=1)
for i in tqdm(noisy_sounds[1:]):
  so,_ = tf.audio.decode_wav(tf.io.read_file(i),desired_channels=1)
  noisy_sounds_list = tf.concat((noisy_sounds_list,so),0)

print(noisy_sounds_list.shape)

batching_size = 36000

noisy_train = []

for i in tqdm(range(0,noisy_sounds_list.shape[0]-batching_size,batching_size)):
    noisy_train.append(noisy_sounds_list[i:i+batching_size])

noisy_train = tf.stack(noisy_train)

print(noisy_train.shape)

  0%|          | 0/99 [00:00<?, ?it/s]

(5938603, 1)


  0%|          | 0/164 [00:00<?, ?it/s]

(164, 36000, 1)


In [14]:
len(noisy_train)

164

In [15]:
test_audio = noisy_train[90]

In [16]:
Audio(np.squeeze(test_audio.numpy()),rate=16000)

In [34]:
# librosa.display.waveshow(np.squeeze(test_audio.numpy()))


In [17]:
# import numpy as np
# from scipy.io.wavfile import write

# rate = 16000
# data = np.squeeze(test_audio.numpy())
# scaled = np.int16(data / np.max(np.abs(data)) * 32767)
# write('audio/noise3.wav', rate, scaled)

In [18]:

Audio(tf.squeeze(model.predict(tf.expand_dims(tf.expand_dims(test_audio,-1),0))),rate=16000)

1/1 [==============================] - 0s 411ms/step


In [35]:

# librosa.display.waveshow(np.squeeze(model.predict(tf.expand_dims(tf.expand_dims(test_audio,-1),0))))
# plt.show()


In [12]:
# import numpy as np
# from scipy.io.wavfile import write

# # rate = 44100
# rate = 16000
# data = tf.squeeze(model.predict(tf.expand_dims(tf.expand_dims(test_audio,-1),0)))
# scaled = np.int16(data / np.max(np.abs(data)) * 32767)
# write('audio/clean3.wav', rate, scaled)

# Inference

In [19]:
batching_size = 12000

def get_audio(path):
    audio,_ = tf.audio.decode_wav(tf.io.read_file(path),1)
    return audio

def inference_preprocess(path):
    audio = get_audio(path)
    audio_len = audio.shape[0]
    batches = []
    for i in range(0,audio_len-batching_size,batching_size):
        batches.append(audio[i:i+batching_size])

    batches.append(audio[-batching_size:])
    diff = audio_len - (i + batching_size)
    return tf.stack(batches), diff

def predict(path):
    test_data,diff = inference_preprocess(path)
    predictions = model.predict(test_data)
    final_op = tf.reshape(predictions[:-1],((predictions.shape[0]-1)*predictions.shape[1],1))
    final_op = tf.concat((final_op,predictions[-1][-diff:]),axis=0)
    return final_op

In [40]:
path = "Noise-Suppression-Using-Deep-Learning/dataset/NoisyData/p226_006.wav"

Audio(np.squeeze(get_audio(path).numpy(),-1),rate=16000)

In [ ]:
Audio(tf.squeeze(predict(noisy_sounds[4])),rate=16000)